In [ ]:
suppressMessages(library(openxlsx))
suppressMessages(library(ggplot2))
suppressMessages(library(ggpubr))
suppressMessages(library(ggrepel))
suppressMessages(library(ggrastr))
suppressMessages(library(reshape2))
suppressMessages(library(lme4))
suppressMessages(library(emmeans))
suppressMessages(library(dplyr))
suppressMessages(library(tidyverse))
suppressMessages(library(Seurat))
suppressMessages(library(foreach))
suppressMessages(library(doParallel))

# SCENIC

## Main (neurons)

In [ ]:
identify_da_regulons <- function(seurat_obj, celltypes, group1, group2) {
  lmer_results <- foreach(celltype = celltypes) %dopar% {
    
    seurat_obj_sub <- subset(seurat_obj, cell.type == celltype & treatment %in% c(group2, group1))
    sample_ids <- unique(seurat_obj_sub$hash.ID)
    
    regulons <- rownames(seurat_obj_sub@assays$regulon)[
      which(apply(seurat_obj_sub@assays$regulon@data, 1, function(x) {
        sum(x != 0) >= (ncol(seurat_obj_sub) * 0.20)
      }))
    ]
    
    regulons <- "Fos..."  # Placeholder to be replaced by actual regulons selection criteria
    
    # Prepare data for modeling
    data <- cbind.data.frame(
      treatment = as.factor(seurat_obj_sub$treatment), 
      sample = as.factor(seurat_obj_sub$hash.ID),
      pool = as.factor(seurat_obj_sub$pool),
      run = as.factor(seurat_obj_sub$run),
      calcr = seurat_obj_sub@assays$SCT@data["Calcr",],
      treatment = factor(seurat_obj_sub$treatment, levels = c(group2, group1))
    )
    
    results <- data.frame(matrix(NA, nrow = length(regulons), ncol = 4))
    colnames(results) <- c("regulon", "beta", "SE", "p_value")
    
    for (i in seq_along(regulons)) {
      set.seed(i)
      data$regulon <- seurat_obj_sub@assays$regulon@data[regulons[i],]
      
      # Linear mixed effects model
      model <- lmer(
        regulon ~ treatment + (1 | sample),
        data = data,
        REML = TRUE
      )
      
      emm <- lsmeans(model, pairwise ~ treatment, adjust = NULL)
      emm_summary <- summary(emm$contrasts)
      
      results$regulon[i] <- regulons[i]
      contrast <- paste0("(", group1, ") - (", group2, ")")
      results$p_value[i] <- emm_summary$p.value[which(emm_summary$contrast == contrast)]
      results$SE[i] <- emm_summary$SE[which(emm_summary$contrast == contrast)]
      results$beta[i] <- emm_summary$estimate[which(emm_summary$contrast == contrast)]
    }
    
    results$celltype <- celltype
    results
  }
  
  names(lmer_results) <- celltypes
  return(lmer_results)
}

process_SCENIC_data <- function(species) {
  # Set parameters based on species
  if (species == "mouse") {
    seurat_obj <- mouse_neurons
    comparison_acute <- "Cagrilintide vs. vehicle (4 hours)"
    comparison_chronic <- "Cagrilintide vs. weight-matched control (8 days)"
  } else if (species == "rat") {
    seurat_obj <- rat_neurons
    comparison_acute <- "Cagrilintide vs. vehicle control (4 hours)"
    comparison_chronic <- "Cagrilintide vs. weight-matched control (8 days)"
  }
  
  # Process acute data
  lmer_acute <- identify_da_regulons(
    seurat_obj = seurat_obj,
    celltypes = c("Glu4.2", "Glu8.0", "Glu8.2"),
    group1 = "A8-A",
    group2 = "V-A"
  )
  lmer_acute <- dplyr::bind_rows(lmer_acute, .id = "celltype")
  lmer_acute$p_adj <- p.adjust(lmer_acute$p_value, method = "fdr")
  lmer_acute$comparison <- comparison_acute
  
  # Process chronic data
  lmer_chronic <- identify_da_regulons(
    seurat_obj = seurat_obj,
    celltypes = c("Glu4.2", "Glu8.0", "Glu8.2"),
    group1 = "A8-C",
    group2 = "WM-C"
  )
  lmer_chronic <- dplyr::bind_rows(lmer_chronic, .id = "celltype")
  lmer_chronic$p_adj <- p.adjust(lmer_chronic$p_value, method = "fdr")
  lmer_chronic$comparison <- comparison_chronic
  
  # Combine acute and chronic data
  lmer_combined <- rbind(lmer_acute, lmer_chronic)
  lmer_combined$celltype <- factor(
    lmer_combined$celltype,
    levels = c("Glu4.2", "Glu8.0", "Glu8.2")
  )
  
  # Add significance symbols
  lmer_combined$p_symbol <- ""
  lmer_combined$p_symbol[lmer_combined$p_adj < 0.05] <- "*"
  lmer_combined$p_symbol[lmer_combined$p_adj < 0.01] <- "**"
  lmer_combined$p_symbol[lmer_combined$p_adj < 0.001] <- "***"
  
  # Add species column
  lmer_combined$species <- species
  
  # Extract time point from comparison
  lmer_combined$time_point <- ifelse(grepl("4 hours", lmer_combined$comparison), "4 hours", "8 days")
  
  return(lmer_combined)
}

In [ ]:
rat.AUC <- read.csv("/projects/perslab/people/jmg776/projects/DVC/output/SCENIC/rat_DVC_neurons_2025/rat_DVC_neurons_2025_auc.csv", header = TRUE, row.names = 1)
mouse.AUC <- read.csv("/projects/perslab/people/jmg776/projects/DVC/output/SCENIC/mouse_DVC_neurons_2025/mouse_DVC_neurons_2025_auc.csv", header = TRUE, row.names = 1)
neurons <- readRDS("/projects/perslab/people/jmg776/projects/DVC/output/Seurat_objs/integrated/neurons_finalized_2025.rds")
neurons$cell.type <- gsub("_(\\d+)_", "\\1.", neurons$cell.type)

In [ ]:
# Set up intermediary objects
mouse_neurons <- subset(neurons, dataset == "mouse.2023")
mouse_neurons[["regulon"]] <- CreateAssayObject(data = t(as.matrix(mouse.AUC)))

rat_neurons <- subset(neurons, species == "rat")
rat_neurons[["regulon"]] <- CreateAssayObject(data = t(as.matrix(rat.AUC)))

In [ ]:
# Process SCENIC data
lmer_mouse <- process_SCENIC_data("mouse")
lmer_rat <- process_SCENIC_data("rat")

lmer_combined <- rbind(lmer_mouse, lmer_rat) %>%
  mutate(
    x = as.numeric(celltype),
    x = ifelse(species == "mouse", x - 0.2, x + 0.2)  # Small offset for subsequent plotting
  )

In [ ]:
signif_data <- lmer_combined %>%  # Adding significance symbols to plot
  filter(p_symbol != "") %>%
  mutate(y = beta + SE + 0.001)

p <- ggplot(lmer_combined, aes(x = celltype, y = beta, color = species, group = species)) +
  geom_point(size = 1, position = position_dodge(width = 0.6)) +
  geom_errorbar(
    aes(ymin = beta - SE, ymax = beta + SE),
    width = 0.1,
    position = position_dodge(width = 0.6)
  ) +
  geom_hline(yintercept = 0, linetype = "dashed", color = "black", size = 0.1) +
  theme_pubr(base_size = 6, base_family = "sans") +
  theme(
    axis.text.x = element_text(angle = 45, hjust = 1),
    legend.position = "top",  # Place legend above the plot
    plot.margin = unit(c(0, 0, 0, 0), "cm"),
    legend.title = element_blank()
  ) +
  ylab(expression(bold(beta))) +
  xlab("") +
  scale_color_manual(values = c("#75C0AF", "#546577"), labels = c("Mouse", "Rat")) +
  facet_wrap(~time_point) +
  geom_text(
    data = signif_data,
    aes(label = p_symbol, y = y),
    size = 2,
    fontface = "bold",
    color = "black",
    position = position_dodge(width = 0.6)
  )

p

## Supplementary (glia)

In [ ]:
identify_da_regulons <- function(seurat_obj, celltypes, group1, group2) {
  lmer_results <- foreach(celltype = celltypes) %dopar% {
    
    seurat_obj_sub <- subset(seurat_obj, cell.type == celltype & treatment %in% c(group2, group1))
    sample_ids <- unique(seurat_obj_sub$hash.ID)
    
    regulons <- rownames(seurat_obj_sub@assays$regulon)[
      which(apply(seurat_obj_sub@assays$regulon@data, 1, function(x) {
        sum(x != 0) >= (ncol(seurat_obj_sub) * 0.20)
      }))
    ]
    
    regulons <- "Fos..."  # Placeholder to be replaced by actual regulons selection criteria
    
    # Prepare data for modeling
    data <- cbind.data.frame(
      treatment = as.factor(seurat_obj_sub$treatment), 
      sample = as.factor(seurat_obj_sub$hash.ID),
      pool = as.factor(seurat_obj_sub$pool),
      run = as.factor(seurat_obj_sub$run),
      calcr = seurat_obj_sub@assays$SCT@data["Calcr",],
      treatment = factor(seurat_obj_sub$treatment, levels = c(group2, group1))
    )
    
    results <- data.frame(matrix(NA, nrow = length(regulons), ncol = 4))
    colnames(results) <- c("regulon", "beta", "SE", "p_value")
    
    for (i in seq_along(regulons)) {
      set.seed(i)
      data$regulon <- seurat_obj_sub@assays$regulon@data[regulons[i],]
      
      # Linear mixed effects model
      model <- lmer(
        regulon ~ treatment + (1 | sample),
        data = data,
        REML = TRUE
      )
      
      emm <- lsmeans(model, pairwise ~ treatment, adjust = NULL)
      emm_summary <- summary(emm$contrasts)
      
      results$regulon[i] <- regulons[i]
      contrast <- paste0("(", group1, ") - (", group2, ")")
      results$p_value[i] <- emm_summary$p.value[which(emm_summary$contrast == contrast)]
      results$SE[i] <- emm_summary$SE[which(emm_summary$contrast == contrast)]
      results$beta[i] <- emm_summary$estimate[which(emm_summary$contrast == contrast)]
    }
    
    results$celltype <- celltype
    results
  }
  
  names(lmer_results) <- celltypes
  return(lmer_results)
}

process_SCENIC_data <- function(species) {
  # Set parameters based on species
  if (species == "mouse") {
    seurat_obj <- mouse_glia
    comparison_acute <- "Cagrilintide vs. vehicle (4 hours)"
    comparison_chronic <- "Cagrilintide vs. weight-matched control (8 days)"
  } else if (species == "rat") {
    seurat_obj <- rat_glia
    comparison_acute <- "Cagrilintide vs. vehicle control (4 hours)"
    comparison_chronic <- "Cagrilintide vs. weight-matched control (8 days)"
  }
  
  # # Process acute data
  # lmer_acute <- identify_da_regulons(
  #   seurat_obj = seurat_obj,
  #   celltypes = c('Ependymal_cells','Astrocytes','OPCs','Oligodendrocytes','Tanycytes','Pericytes','VLMCs','Microglia','Endothelial_cells'),
  #   group1 = "A8-A",
  #   group2 = "V-A"
  # )
  # lmer_acute <- dplyr::bind_rows(lmer_acute, .id = "celltype")
  # lmer_acute$p_adj <- p.adjust(lmer_acute$p_value, method = "fdr")
  # lmer_acute$comparison <- comparison_acute
  
  # Process chronic data
  lmer_chronic <- identify_da_regulons(
    seurat_obj = seurat_obj,
    celltypes = c('Ependymal_cells','Astrocytes','OPCs','Oligodendrocytes','Tanycytes','Pericytes','VLMCs','Microglia','Endothelial_cells'),
    group1 = "A8-C",
    group2 = "WM-C"
  )
  lmer_chronic <- dplyr::bind_rows(lmer_chronic, .id = "celltype")
  lmer_chronic$p_adj <- p.adjust(lmer_chronic$p_value, method = "fdr")
  lmer_chronic$comparison <- comparison_chronic
  
  # Combine acute and chronic data
  lmer_combined <- rbind(lmer_acute, lmer_chronic)
  lmer_combined$celltype <- factor(
    lmer_combined$celltype,
    levels = c('Ependymal_cells','Astrocytes','OPCs','Oligodendrocytes','Tanycytes','Pericytes','VLMCs','Microglia','Endothelial_cells')
  )
  
  # Add significance symbols
  lmer_combined$p_symbol <- ""
  lmer_combined$p_symbol[lmer_combined$p_adj < 0.05] <- "*"
  lmer_combined$p_symbol[lmer_combined$p_adj < 0.01] <- "**"
  lmer_combined$p_symbol[lmer_combined$p_adj < 0.001] <- "***"
  
  # Add species column
  lmer_combined$species <- species
  
  # Extract time point from comparison
  lmer_combined$time_point <- ifelse(grepl("4 hours", lmer_combined$comparison), "4 hours", "8 days")
  
  return(lmer_chronic)
}

In [ ]:
rat.AUC <- read.csv("/projects/perslab/people/jmg776/projects/DVC/output/SCENIC/rat_DVC_glia_2025/rat_DVC_glia_2025_auc.csv", header = TRUE, row.names = 1)
mouse.AUC <- read.csv("/projects/perslab/people/jmg776/projects/DVC/output/SCENIC/mouse_DVC_glia_2025/mouse_DVC_glia_2025_auc.csv", header = TRUE, row.names = 1)
glia <- readRDS("/projects/perslab/people/jmg776/projects/DVC/output/Seurat_objs/integrated/glia_finalized_2025.rds")

In [ ]:
# Set up intermediary objects
mouse_glia <- subset(glia, dataset == "mouse.2023")
mouse_glia[["regulon"]] <- CreateAssayObject(data = t(as.matrix(mouse.AUC)))

rat_glia <- subset(glia, species == "rat")
rat_glia[["regulon"]] <- CreateAssayObject(data = t(as.matrix(rat.AUC)))

In [ ]:
# Process SCENIC data
lmer_mouse <- process_SCENIC_data("mouse")
lmer_rat <- process_SCENIC_data("rat")

lmer_combined <- rbind(lmer_mouse, lmer_rat) %>%
  mutate(
    x = as.numeric(celltype),
    x = ifelse(species == "mouse", x - 0.2, x + 0.2)  # Small offset for subsequent plotting
  )

In [ ]:
signif_data <- lmer_combined %>%  # Adding significance symbols to plot
  filter(p_symbol != "") %>%
  mutate(y = beta + SE + 0.001)

p <- ggplot(lmer_combined, aes(x = celltype, y = beta, color = species, group = species)) +
  geom_point(size = 1, position = position_dodge(width = 0.6)) +
  geom_errorbar(
    aes(ymin = beta - SE, ymax = beta + SE),
    width = 0.1,
    position = position_dodge(width = 0.6)
  ) +
  geom_hline(yintercept = 0, linetype = "dashed", color = "black", size = 0.1) +
  theme_pubr(base_size = 6, base_family = "sans") +
  theme(
    axis.text.x = element_text(angle = 45, hjust = 1),
    legend.position = "top",  # Place legend above the plot
    plot.margin = unit(c(0, 0, 0, 0), "cm"),
    legend.title = element_blank()
  ) +
  ylab(expression(bold(beta))) +
  xlab("") +
  scale_color_manual(values = c("#75C0AF", "#546577"), labels = c("Mouse", "Rat")) +
  facet_wrap(~time_point) +
  geom_text(
    data = signif_data,
    aes(label = p_symbol, y = y),
    size = 2,
    fontface = "bold",
    color = "black",
    position = position_dodge(width = 0.6)
  )

p

# ggsave("/projects/perslab/people/jmg776/projects/DVC/figures/2025/figure_supp_SCENIC_glia.pdf", width = 10, height = 10)